## ENV SETUP

1. Install uv (or do it you're own way)
2. Run `uv sync`
3. Run `source .venv/bin/activate`

You're good to go.

# Instructions

The Task : Create the best CadQuery code generator model. 

1. Load the dataset (147K pairs of Images/CadQuery code).
2. Create a baseline model and evaluate it with the given metrics.
3. Enhance by any manner the baseline model and evaluate it again.
4. Explain you choices and possible bottlenecks. 
5. Show what enhancements you would have done if you had more time.

You can do *WHATEVER* you want, be creative, result is not what matters the most. 
Creating new model architectures, reusing ones you used in the past, fine-tuning, etc...

If you are GPU poor, there are solutions. Absolute value is not what matters, relative value between baseline and enhanced model is what matters.

In [2]:
from datasets import load_dataset

ds = load_dataset("CADCODER/GenCAD-Code", split=["train", "test"], num_proc=4)
train_ds, test_ds = ds


c:\Users\PC\anaconda3\envs\crewai-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(train_ds[0])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=448x448 at 0x1A3650456C0>, 'deepcad_id': '0000/00006371', 'cadquery': 'import cadquery as cq\n# Generating a workplane for sketch 0\nwp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(-0.015625, -0.0078125, 0.0), cq.Vector(1.0, 0.0, 0.0), cq.Vector(0.0, 0.0, 1.0)))\nloop0=wp_sketch0.moveTo(0.0, 0.0).threePointArc((0.0007948582418457166, -0.0019189575476279677), (0.0027138157894736844, -0.0027138157894736844)).lineTo(0.021217105263157895, -0.0027138157894736844).threePointArc((0.022787161438489866, -0.00206347722796355), (0.0234375, -0.000493421052631579)).lineTo(0.0234375, 0.018256578947368422).threePointArc((0.02283825686147997, 0.019949990385858287), (0.021217105263157895, 0.020723684210526318)).lineTo(0.0022203947368421052, 0.020723684210526318).threePointArc((0.0005992431385200307, 0.019949990385858287), (0.0, 0.018256578947368422)).lineTo(0.0, 0.0).close()\nsolid0=wp_sketch0.add(loop0).extrude(0.75)\nsolid=solid0\n', 'token_

In [4]:
train_ds[8]['image'].show()

In [6]:
!pip install transformers datasets

  Using cached transformers-4.53.0-py3-none-any.whl.metadata (39 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
Using cached transformers-4.53.0-py3-none-any.whl (10.8 MB)
Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl (2.5 MB)
Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)

  Attempting uninstall: tokenizers

    Found existing installation: tokenizers 0.20.3

    Uninstalling tokenizers-0.20.3:

      Successfully uninstalled tokenizers-0.20.3

   ------------- -------------------------- 1/3 [tokenizers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- ------------- 2/3 [transformers]
   -------------------------- -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.2 which is incompatible.


In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/216.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/216.0 MB 3.3 MB/s eta 0:01:05
   ---------------------------------------- 1.3/216.0 MB 4.8 MB/s eta 0:00:45
    --------------------------------------- 2.9/216.0 MB 4.7 MB/s eta 0:00:46
    --------------------------------------- 4.2/216.0 MB 5.3 MB/s eta 0:00:40
   - -------------------------------------- 5.5/216.0 MB 6.0 MB/s eta 0:00:36
   - -------------------------------------- 7.1/216.0 MB 5.7 MB/s eta 0:00:37
   - -------------------------------------- 8.7/216.0 MB 6.1 MB/s eta 0:00:34
   - -------------------------------------- 10.0/216.0 MB 6.3 MB/s eta 0:00:33
   -- ------------------------------------- 11.0/216.0 MB 6.1 MB/s eta 0:00:34
   -- ------------------------------------- 12.1/216.0 MB 5.9 MB/s eta 0:00:35
   -- ------------------------------------- 13.1/216.0 MB 5.8 MB/s eta 0:00:35
   -- ----

In [9]:
!pip install tqdm pillow

In [11]:
print(ds[0].keys())

dict_keys(['image', 'deepcad_id', 'cadquery', 'token_count', 'prompt', 'hundred_subset'])


In [12]:
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
from tqdm import tqdm

# Charger le dataset
ds = load_dataset("CADCODER/GenCAD-Code", split="train[:500]")  # Pour aller vite on prend 500 exemples
query_ds = load_dataset("CADCODER/GenCAD-Code", split="test[:10]")  # Juste 10 pour test

# Charger CLIP
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

def get_embedding(image):
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    return outputs[0]

# Indexation des images du dataset
train_embeddings = []
train_codes = []

for sample in tqdm(ds):
    emb = get_embedding(sample['image'])
    train_embeddings.append(emb)
    train_codes.append(sample['cadquery'])

# Baseline Retrieval sur les images de test
for query in query_ds:
    query_emb = get_embedding(query['image'])

    # Calcul de la similarité cosine
    similarities = [torch.nn.functional.cosine_similarity(query_emb, e, dim=0) for e in train_embeddings]
    best_match_idx = torch.argmax(torch.tensor(similarities))

    retrieved_code = train_codes[best_match_idx]
    true_code = query['cadquery']

    print("🖼️ Query image code:")
    print(true_code)
    print("\n🔁 Retrieved code from most similar image:")
    print(retrieved_code)
    print("----------------------------------------------------")



100%|██████████| 500/500 [00:46<00:00, 10.71it/s]


🖼️ Query image code:
import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(0.0, -0.75, -0.75), cq.Vector(3.749399456654644e-33, 1.0, -6.123233995736766e-17), cq.Vector(1.0, 0.0, 6.123233995736766e-17)))
loop0=wp_sketch0.moveTo(1.5, 0.0).lineTo(1.5, 1.5).lineTo(0.0, 1.5).lineTo(0.0, 0.0).close()
loop1=wp_sketch0.moveTo(0.7578947368421053, 0.5368421052631579).circle(0.14210526315789473)
loop2=wp_sketch0.moveTo(0.7578947368421053, 0.9315789473684211).circle(0.14210526315789473)
solid0=wp_sketch0.add(loop0).add(loop1).add(loop2).extrude(0.03125)
solid=solid0


🔁 Retrieved code from most similar image:
import cadquery as cq
# Generating a workplane for sketch 0
wp_sketch0 = cq.Workplane(cq.Plane(cq.Vector(-0.453125, 0.0, -0.75), cq.Vector(1.0, 6.123233995736766e-17, -6.123233995736766e-17), cq.Vector(6.123233995736766e-17, -1.0, 6.123233995736766e-17)))
loop0=wp_sketch0.moveTo(0.9, 0.0).lineTo(0.9, 1.5).lineTo(0.0, 1.5).lineTo(0.0, 0.0).clo

In [14]:
!pip install cadquery

  Using cached cadquery-2.5.2-py3-none-any.whl.metadata (16 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached typish-1.9.3-py3-none-any.whl.metadata (7.2 kB)
  Using cached path-17.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached cadquery-2.5.2-py3-none-any.whl (163 kB)
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/90.9 MB ? eta -:

In [16]:
!pip install trimesh

   ---------------------------------------- 0.0/712.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/712.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   -------------- ------------------------- 262.1/712.0 kB ? eta -:--:--
   --------------------------- ---------- 524.3/712.0 kB 129.1 kB/s eta 0:00:02
   --------------------------- ---------- 524.3/71

In [17]:
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best

In [26]:
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best
import cadquery as cq

iou_scores = []
syntax_results = []

# Vérifie que le code peut s'exécuter avec CadQuery
def is_valid_code(code_str):
    try:
        local_env = {}
        exec(code_str, {"cq": cq}, local_env)
        return True
    except:
        return False

for query in tqdm(query_ds):
    query_emb = get_embedding(query['image'])

    similarities = [torch.nn.functional.cosine_similarity(query_emb, e, dim=0) for e in train_embeddings]
    best_match_idx = torch.argmax(torch.tensor(similarities))

    retrieved_code = train_codes[best_match_idx]
    true_code = query['cadquery']  # ou 'cadquery', selon ton dataset

    # Vérifie que les deux codes sont valides AVANT d’évaluer
    if is_valid_code(retrieved_code) and is_valid_code(true_code):
        codes = {
            "sample_code": retrieved_code,
            "sample_code_2": true_code
        }

        try:
            vsr = evaluate_syntax_rate_simple(codes)
            iou = get_iou_best(retrieved_code, true_code)
        except Exception as e:
            vsr = 0
            iou = 0
    else:
        vsr = 0
        iou = 0

    syntax_results.append(vsr)
    iou_scores.append(iou)

# Résumé des résultats
print("✅ Moyenne Valid Syntax Rate :", round(sum(syntax_results) / len(syntax_results), 3))
print("📐 Moyenne IOU :", round(sum(iou_scores) / len(iou_scores), 3))


100%|██████████| 10/10 [00:30<00:00,  3.05s/it]

✅ Moyenne Valid Syntax Rate : 0.0
📐 Moyenne IOU : 0.0


In [27]:
print(query.keys())

dict_keys(['image', 'deepcad_id', 'cadquery', 'token_count', 'prompt', 'hundred_subset'])


In [1]:
from datasets import load_dataset
ds = load_dataset("CADCODER/GenCAD-Code", num_proc=16, split=["train", "test"], cache_dir="/Volumes/BIG-DATA/HUGGINGFACE_CACHE")

/Users/oweindourneau/projects/mecagent-technical-test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Evaluation Metrics

1. Valid Syntax Rate metric assess the validity of the code by executing and checking if error are returned.
2. Best IOU assess the similarity between the meshes generated by the code.

In [2]:
from metrics.valid_syntax_rate import evaluate_syntax_rate_simple
from metrics.best_iou import get_iou_best

In [30]:
## Example usage of the metrics
sample_code = """
height = 60.0
width = 80.0
thickness = 10.0
diameter = 22.0

# make the base
result = (
    cq.Workplane("XY")
    .box(height, width, thickness)
)
"""

sample_code_2 = """
 height = 60.0
 width = 80.0
 thickness = 10.0
 diameter = 22.0
 padding = 12.0

 # make the base
 result = (
     cq.Workplane("XY")
     .box(height, width, thickness)
     .faces(">Z")
     .workplane()
     .hole(diameter)
     .faces(">Z")
     .workplane()
     .rect(height - padding, width - padding, forConstruction=True)
     .vertices()
     .cboreHole(2.4, 4.4, 2.1)
 )
"""
codes = {
    "sample_code": sample_code,
    "sample_code_2": sample_code_2,
}
vsr = evaluate_syntax_rate_simple(codes)
print("Valid Syntax Rate:", vsr)
iou = get_iou_best(sample_code, sample_code_2)
print("IOU:", iou)

Valid Syntax Rate: 1.0


ModuleNotFoundError: No module named 'scipy'

In [29]:
!pip install scipy

## Have Fun